<a href="https://colab.research.google.com/github/jpsn2/Projeto_Aprendizagem_de_Maquina_2025_2/blob/main/notebooks/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Leitura do dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/dataset_project_AM/yeast.data',
        sep=' ',
        engine="python",    # necessário para autodetectar separador
        comment="#",        # ignora linhas que começam com #
        header=None,        # caso não haja cabeçalho
        encoding="utf-8",   # padrão moderno
        on_bad_lines="skip" # ignora linhas com problemas
    )

try:
    print(df.head())
except Exception as e:
    print("Erro ao ler arquivo:", e)

           0   1     2   3     4   5     6   7     8   9    10  11   12  13  \
0  ADT1_YEAST NaN  0.58 NaN  0.61 NaN  0.47 NaN  0.13 NaN  0.5 NaN  0.0 NaN   
1  ADT2_YEAST NaN  0.43 NaN  0.67 NaN  0.48 NaN  0.27 NaN  0.5 NaN  0.0 NaN   
2  ADT3_YEAST NaN  0.64 NaN  0.62 NaN  0.49 NaN  0.15 NaN  0.5 NaN  0.0 NaN   
3  AAR2_YEAST NaN  0.58 NaN  0.44 NaN  0.57 NaN  0.13 NaN  0.5 NaN  0.0 NaN   
4  AATM_YEAST NaN  0.42 NaN  0.44 NaN  0.48 NaN  0.54 NaN  0.5 NaN  0.0 NaN   

     14  15    16  17   18  
0  0.48 NaN  0.22 NaN  MIT  
1  0.53 NaN  0.22 NaN  MIT  
2  0.53 NaN  0.22 NaN  MIT  
3  0.54 NaN  0.22 NaN  NUC  
4  0.48 NaN  0.22 NaN  MIT  


## Verificação da quantidade de linhas e colunas

In [4]:
df.shape

(1418, 19)

## Drop de colunas com valores NaN

In [5]:
df = df.dropna(axis=1, how='all')

In [6]:
df.head()

,0,2,4,6,8,10,12,14,16,18
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


## Renomeando as colunas, e retirando as antigas sem nome

In [7]:
df['mcg'] = df[2]
df['gvh'] = df[4]
df['alm'] = df[6]
df['mit'] = df[8]
df['erl'] = df[10]
df['pox'] = df[12]
df['vac'] = df[14]
df['nuc'] = df[16]
df['Localization site'] = df[18]

In [8]:
df = df.drop(columns=[0])
df = df.drop(columns=[2])
df = df.drop(columns=[4])
df = df.drop(columns=[6])
df = df.drop(columns=[8])
df = df.drop(columns=[10])
df = df.drop(columns=[12])
df = df.drop(columns=[14])
df = df.drop(columns=[16])
df = df.drop(columns=[18])

In [9]:
df.head()

,mcg,gvh,alm,mit,erl,pox,vac,nuc,Localization site
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [10]:
df.shape

(1418, 9)